In [4]:
pip install lime

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: python-dateutil>=2.1 in c:\users\fannie\anaconda3\lib\site-packages (from matplotlib->lime) (2.8.1)
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283850 sha256=3808de26ddf3ce8305f3f3f5e7550a4d91e43e79f078c737491fa2ee1aaaf1ef
  Stored in directory: c:\users\fannie\appdata\local\pip\cache\wheels\e6\a6\20\cc1e293fcdb67ede666fed293cb895395e7ecceb4467779546
Successfully built lime



In [6]:
pip install transformers

  Attempting uninstall: packaging
    Found existing installation: packaging 19.1
    Uninstalling packaging-19.1:
      Successfully uninstalled packaging-19.1
Note: you may need to restart the kernel to use updated packages.


ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

liwc-text-analysis 1.0.2 requires packaging==19.1, but you'll have packaging 21.3 which is incompatible.


In [8]:
pip install example --use-feature=2020-resolver

  Created wheel for example: filename=example-0.1.0-py3-none-any.whl size=1243 sha256=add982510ed6af73fa72f5852e7a55106b4f327a30f2411617286a6c79014ecb
  Stored in directory: c:\users\fannie\appdata\local\pip\cache\wheels\0f\24\d2\9282c61f558dc58e67735404c49c03d873555dd867ddbc674a
Successfully built example
Note: you may need to restart the kernel to use updated packages.


In [30]:
## for data
import json
import pandas as pd
import numpy as np
## for plotting
import matplotlib.pyplot as plt
import seaborn as sns
## for processing
import re
import nltk
## for bag-of-words
from sklearn import feature_extraction, model_selection, naive_bayes, pipeline, manifold, preprocessing,feature_selection
## for explainer
from lime import lime_text
## for word embedding
import gensim
import gensim.downloader as gensim_api
## for deep learning
from tensorflow.keras import models, layers, preprocessing as kprocessing
from tensorflow.keras import backend as K
## for bert language model
import transformers

In [57]:
from sklearn import feature_selection
from sklearn import metrics

In [31]:
review_sample=pd.read_csv('review_sample.csv')

In [16]:
def utils_preprocess_text(text, flg_stemm=False, flg_lemm=True, lst_stopwords=None):
    ## clean (convert to lowercase and remove punctuations and   characters and then strip)
    text = re.sub(r'[^\w\s]', '',str(text).lower().strip())
            
    ## Tokenize (convert from string to list)
    lst_text = text.split()
    ## remove Stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in 
                    lst_stopwords]
                
    ## Stemming (remove -ing, -ly, ...)
    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]
                
    ## Lemmatisation (convert the word into root word)
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]
            
    ## back to string from list
    text = " ".join(lst_text)
    return text

In [11]:
lst_stopwords = nltk.corpus.stopwords.words("english")
lst_stopwords

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [40]:
review_sample["text_clean"] = review_sample["text"].apply(lambda x: 
          utils_preprocess_text(x, flg_stemm=False, flg_lemm=True, 
          lst_stopwords=lst_stopwords))
review_sample.at[5,'text_clean']

'conduct sort business company overcharged system wasnt even installed properly asked remove additional component werent even installed properly refused take item shoddy workmanship way overpriced extremely poor customer service'

In [41]:
## Count (classic BoW)
vectorizer = feature_extraction.text.CountVectorizer(max_features=10000, ngram_range=(1,1))

## Tf-Idf (advanced variant of BoW)
vectorizer = feature_extraction.text.TfidfVectorizer(max_features=10000, ngram_range=(1,1))

In [42]:
## split dataset
dtf_train, dtf_test = model_selection.train_test_split(review_sample, test_size=0.3)
## get target
y_train = dtf_train["useful_level"].values
y_test = dtf_test["useful_level"].values

In [46]:
corpus = dtf_train["text_clean"]
vectorizer.fit(corpus)
X_train = vectorizer.transform(corpus)
dic_vocabulary = vectorizer.vocabulary_

In [59]:
X_train.shape

(210, 4845)

In [47]:
dic_vocabulary

{'cant': 714,
 'understand': 4510,
 'hot': 2134,
 'yoga': 4828,
 'studio': 4131,
 'carpet': 739,
 'good': 1899,
 'class': 873,
 'got': 1905,
 'hear': 2038,
 'pumping': 3346,
 'liked': 2475,
 'posture': 3245,
 'sequence': 3773,
 'lot': 2537,
 'get': 1855,
 'mat': 2633,
 'point': 3206,
 'uncomfortable': 4502,
 'soggy': 3958,
 'thing': 4318,
 'id': 2170,
 'go': 1888,
 'back': 399,
 'switched': 4198,
 'hard': 2015,
 'wood': 4766,
 'floor': 1713,
 'could': 1056,
 'cleaned': 878,
 'home': 2103,
 'away': 389,
 'staff': 4054,
 'personable': 3126,
 'nice': 2844,
 'wifi': 4731,
 'close': 892,
 'great': 1925,
 'selection': 3756,
 'life': 2469,
 'black': 527,
 'woman': 4758,
 'hair': 1986,
 'serious': 3777,
 'business': 667,
 'turn': 4472,
 'make': 2577,
 'finding': 1675,
 'decent': 1198,
 'beauty': 466,
 'supply': 4167,
 'store': 4109,
 'kind': 2370,
 'big': 510,
 'deal': 1191,
 'unfortunately': 4521,
 'cambridge': 700,
 'boston': 575,
 'proper': 3322,
 'sorely': 3979,
 'lacking': 2401,
 'scene':

Try below codes with more data

y = dtf_train["useful_level"]
X_names = vectorizer.get_feature_names()
p_value_limit = 1
dtf_features = pd.DataFrame()
for cat in np.unique(y):
    chi2, p = feature_selection.chi2(X_train, y==cat)
    dtf_features = dtf_features.append(pd.DataFrame(
                   {"feature":X_names, "score":1-p, "y":cat}))
    dtf_features = dtf_features.sort_values(["y","score"], 
                    ascending=[True,False])
    dtf_features = dtf_features[dtf_features["score"]>p_value_limit]
X_names = dtf_features["feature"].unique().tolist()

for cat in np.unique(y):
    print("# {}:".format(cat))
    print("  . selected features:",
         len(dtf_features[dtf_features["y"]==cat]))
    print("  . top features:", ",".join(
dtf_features[dtf_features["y"]==cat]["feature"].values[:10]))
    print(" ")

vectorizer = feature_extraction.text.TfidfVectorizer(vocabulary=X_names)
vectorizer.fit(corpus)
X_train = vectorizer.transform(corpus)
dic_vocabulary = vectorizer.vocabulary_

Try classifier

In [53]:
classifier = naive_bayes.MultinomialNB()

In [54]:
## pipeline
model = pipeline.Pipeline([("vectorizer", vectorizer),  
                           ("classifier", classifier)])
## train classifier
model["classifier"].fit(X_train, y_train)
## test
X_test = dtf_test["text_clean"].values
predicted = model.predict(X_test)
predicted_prob = model.predict_proba(X_test)

In [58]:
## Accuracy, Precision, Recall
accuracy = metrics.accuracy_score(y_test, predicted)
auc = metrics.roc_auc_score(y_test, predicted_prob, 
                            multi_class="ovr")
print("Accuracy:",  round(accuracy,2))
print("Auc:", round(auc,2))
print("Detail:")
print(metrics.classification_report(y_test, predicted))

Accuracy: 0.33
Auc: 0.55
Detail:
              precision    recall  f1-score   support

         1.0       0.33      1.00      0.49        27
         2.0       0.00      0.00      0.00        21
         3.0       0.43      0.12      0.19        25
         4.0       0.00      0.00      0.00        17

    accuracy                           0.33        90
   macro avg       0.19      0.28      0.17        90
weighted avg       0.22      0.33      0.20        90



C:\Users\Fannie\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Word2Vec

In [60]:
nlp = gensim_api.load("word2vec-google-news-300")

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [94]:
corpus = dtf_train["text_clean"]

## create list of lists of unigrams
lst_corpus = []
for string in corpus:
    lst_words = string.split() 
    lst_grams = [" ".join(lst_words[i:i+1]) for i in range(0, len(lst_words), 1)]
    lst_corpus.append(lst_grams)

In [95]:
nlp = gensim.models.word2vec.Word2Vec(lst_corpus, vector_size=300,  window=8, min_count=1, sg=1)

In [96]:
## tokenize text
tokenizer = kprocessing.text.Tokenizer(lower=True, split=' ', 
                     oov_token="NaN", 
                     filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(lst_corpus)
dic_vocabulary = tokenizer.word_index
## create sequence
lst_text2seq= tokenizer.texts_to_sequences(lst_corpus)
## padding sequence
X_train = kprocessing.sequence.pad_sequences(lst_text2seq, 
                    maxlen=100, padding="post", truncating="post")

In [97]:
i = 0

## list of text: ["I like this", ...]
len_txt = len(dtf_train["text_clean"].iloc[i].split())
print("from: ", dtf_train["text_clean"].iloc[i], "| len:", len_txt)

## sequence of token ids: [[1, 2, 3], ...]
len_tokens = len(X_train[i])
print("to: ", X_train[i], "| len:", len(X_train[i]))

## vocabulary: 
print("check: ", dtf_train["text_clean"].iloc[i].split()[0], 
      " -- idx in vocabulary -->", 
      dic_vocabulary[dtf_train["text_clean"].iloc[i].split()[0]])

print("vocabulary: ", dict(list(dic_vocabulary.items())[0:5]), "... (padding element, 0)")

from:  cant understand hot yoga studio carpet good class got hear pumping liked posture sequence lot get mat point uncomfortable soggy carpet thing id go back switched hard wood floor could cleaned class | len: 32
to:  [ 112  342  388  636 1143  499    3  164   24  343 1144  246 1553 2282
   80    9 2283  210  898 1145  499   43  155   30   12 2284  247  899
  437   34  900  164    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0] | len: 100
check:  cant  -- idx in vocabulary --> 112
vocabulary:  {'NaN': 1, 'place': 2, 'good': 3, 'one': 4, 'time': 5} ... (padding element, 0)


In [99]:
corpus = dtf_test["text_clean"]

## create list of n-grams
lst_corpus = []
for string in corpus:
    lst_words = string.split()
    lst_grams = [" ".join(lst_words[i:i+1]) for i in range(0, 
                 len(lst_words), 1)]
    lst_corpus.append(lst_grams)

In [100]:
## text to sequence with the fitted tokenizer
lst_text2seq = tokenizer.texts_to_sequences(lst_corpus)

## padding sequence
X_test = kprocessing.sequence.pad_sequences(lst_text2seq, maxlen=100,
             padding="post", truncating="post")

In [101]:
## start the matrix (length of vocabulary x vector size) with all 0s
embeddings = np.zeros((len(dic_vocabulary)+1, 300))
for word,idx in dic_vocabulary.items():
    ## update the row with vector
    try:
        embeddings[idx] =  nlp[word]
    ## if word not in model then skip and the row stays all 0s
    except:
        pass

In [98]:
X_train.shape

(210, 100)

In [102]:
X_test.shape

(90, 100)

In [103]:
## start the matrix (length of vocabulary x vector size) with all 0s
embeddings = np.zeros((len(dic_vocabulary)+1, 300))
for word,idx in dic_vocabulary.items():
    ## update the row with vector
    try:
        embeddings[idx] =  nlp[word]
    ## if word not in model then skip and the row stays all 0s
    except:
        pass

In [104]:
word = "data"
print("dic[word]:", dic_vocabulary[word], "|idx")
print("embeddings[idx]:", embeddings[dic_vocabulary[word]].shape, 
      "|vector")

dic[word]: 1919 |idx
embeddings[idx]: (300,) |vector
